In [4]:
from os import listdir
from os.path import isfile, join

import pickle
import numpy as np
import scipy as sp
from tensorflow.keras.models import load_model
import pandas as pd
from sklearn.neighbors import DistanceMetric

from benchmark.cfg import OHConverter

from constants.var_types import VAR_TYPES

In [5]:
import os
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")


No GPU found


In [ ]:
# Custom Realistic Scores

In [8]:
def internetadv_ratio_contraint(dsName, df_oh, df_cf, df_fc, cat_feats, num_feats, model, converter):
    df_cf_found, df_fc_found = get_cleaned_processed(df_cf, df_fc, cat_feats, model, converter)
    if df_cf_found.shape[0] > 0:
        scaler = load_scaler(dsName)
        df_num_scaled = pd.DataFrame(scaler.inverse_transform(df_cf_found[['1555', '1556', '1557']]))
        
        output = ((df_num_scaled[1]/df_num_scaled[0]).map(lambda x: round(x, 2))-df_num_scaled[2].map(lambda x: round(x, 2)) < 0.01).map(int)
        output.index = df_cf_found.index
        
        return output
    
    return np.nan

In [9]:
def pbc_multivariate_constraints(dsName, df_oh, df_cf, df_fc, cat_feats, num_feats, model, converter):
    df_cf_found, df_fc_found = get_cleaned_processed(df_cf, df_fc, cat_feats, model, converter)
    if df_cf_found.shape[0] > 0:
        scaler = load_scaler(dsName)
        cf_scaled = pd.DataFrame(scaler.inverse_transform(df_cf_found))
        
        x1 = (cf_scaled[2].map(lambda x: round(x, 4))==(cf_scaled[0]*cf_scaled[1]).map(lambda x: round(x, 4))).map(int)
        
        x2 = (cf_scaled[3].map(lambda x: round(x, 3))-(cf_scaled[1]/cf_scaled[0]).map(lambda x: round(x, 3))<0.01).map(int)
        
        x3 = (cf_scaled[4].map(lambda x: round(x, 3))-(cf_scaled[7]/cf_scaled[2]).map(lambda x: round(x, 3))<0.01).map(int)
        
        x4 = (cf_scaled[5].map(lambda x: round(x, 3))-(cf_scaled[8]/cf_scaled[2]).map(lambda x: round(x, 3))<0.01).map(int)
        
        x5 = (cf_scaled[6].map(lambda x: round(x, 3))-(cf_scaled[7]/cf_scaled[9]).map(lambda x: round(x, 3))<0.01).map(int)
        
        output = (x1+x2+x3+x4+x5).map(lambda x : x ==5).map(int)
        output.index = df_cf_found.index
        
        return output
    
    return np.nan

In [10]:
def bcw_radius_area_constraint(dsName, df_oh, df_cf, df_fc, cat_feats, num_feats, model, converter):
    df_cf_found, df_fc_found = get_cleaned_processed(df_cf, df_fc, cat_feats, model, converter)
    if df_cf_found.shape[0] > 0:
        scaler = load_scaler(dsName)
        cf_scaled = pd.DataFrame(scaler.inverse_transform(df_cf_found))
        
        output = (cf_scaled[1]**2*np.pi-cf_scaled[4]<cf_scaled[4]*0.10).map(int)
        output.index = df_cf_found.index
        
        return output
    
    return np.nan

In [11]:
dict_functions = {
    'BCW': {'ruc': [], 'rmc': [bcw_radius_area_constraint]},
    'PBC': {'ruc': [], 'rmc': [pbc_multivariate_constraints]},
    'InternetAdv': {'ruc': [], 'rmc': [internetadv_ratio_contraint]}
}

In [12]:
# Verify if offset between the x and o classes is, at maximum 1
def tictactoe_constraint(dsName, df_oh, df_cf, df_fc, cat_feats, num_feats, model, converter):
    df_cf_found, df_fc_found = get_cleaned_processed(df_cf, df_fc, cat_feats, model, converter)
    if df_cf_found.shape[0] > 0:
        if cat_feats:
            
            df_ohi = check_one_hot_integrity(df_oh, df_cf, df_fc, cat_feats, num_feats, model, converter)
            df_cb = check_binary_categorical(df_oh, df_cf, df_fc, cat_feats, num_feats, model, converter)
            
            dfc_converted_array = []
            for idx in range(df_cf_found.shape[0]):
                if df_ohi.iloc[idx] == 1 and df_cb.iloc[idx] == 1:
                    dfc_converted_array.append(converter.convert(df_cf_found.iloc[int(idx):int(idx)+1].to_numpy())[0])
                else:
                    dfc_converted_array.append(['x']*df_cf_found.shape[1])
            dfc_converted = pd.DataFrame(dfc_converted_array)
            
            dict_ttt = {'x': 1, 'o': -1, 'b': 0}
            for col in list(dfc_converted.columns):
                dfc_converted[col] = dfc_converted[col].apply(lambda x: dict_ttt[x])
            
            output = dfc_converted.sum(axis=1).map(lambda x: int(x in [0, 1, -1]))
            output.index = df_cf_found.index
                
            return output
    
    return np.nan
    

In [ ]:
# Realistic Scores

In [14]:
def check_outside_num_range(dsName, df_oh, df_cf, df_fc, cat_feats, num_feats, model, converter):
    df_cf_found, df_fc_found = get_cleaned_processed(df_cf, df_fc, cat_feats, model, converter)
    if df_cf_found.shape[0] > 0:
        if len(num_feats) > 0:
            scaler = load_scaler(dsName)
            scaled_ds = pd.DataFrame(scaler.inverse_transform(df_oh[num_feats]))

            num_feats_idx = []

            for n_f in num_feats:
                num_feats_idx.append(str(list(df_oh.columns).index(n_f)))
            scaled_cf = pd.DataFrame(scaler.inverse_transform(df_cf_found[num_feats_idx]))

            for i in range(scaled_ds.shape[1]):
                scaled_cf[i] = scaled_cf[i].map(lambda x: scaled_ds[i].min() <= x and x <= scaled_ds[i].max())

            output = scaled_cf.T.sum().map(lambda x: x == scaled_cf.shape[1]).map(int)
            output.index = df_cf_found.index
            
            
            return output
    return np.nan

In [15]:
def load_scaler(dsName):
    return pickle.load(open(f'./scalers/{dsName}.sc', 'rb'))

In [16]:
def verify_one_hot(df_oh, df_cf, df_fc, cat_feats, num_feats, model, converter):
    oh_i = check_one_hot_integrity(df_oh, df_cf, df_fc, cat_feats, num_feats, model, converter)
    oh_b = check_binary_categorical(df_oh, df_cf, df_fc, cat_feats, num_feats, model, converter)
    
    return (oh_i*oh_b).sum()/oh_b.shape[0]

In [17]:
def check_one_hot_integrity(df_oh, df_cf, df_fc, cat_feats, num_feats, model, converter):
    df_cf_found, df_fc_found = get_cleaned_processed(df_cf, df_fc, cat_feats, model, converter)
    bin_feats = converter.binary_cats
        
    if df_cf_found.shape[0] > 0:
        if len(cat_feats) > 0:
            cat_oh_feats = list(set(cat_feats)-set(bin_feats))

            if len(cat_oh_feats) > 0:
                array_cat_int = []
                for cat_oh_f in cat_oh_feats:
                    array_cat_int.append(df_cf_found.iloc[:, list(converter.dict_feat_idx[cat_oh_f].values())].apply(lambda x: x>0, axis=1).sum(axis=1).map(lambda x: x == 1))

                df_cat_int = pd.concat(array_cat_int, axis=1)

                output = df_cat_int.sum(axis=1).map(lambda x : x==df_cat_int.shape[1]).map(int)
                output.index = df_cf_found.index
                
                return output
        
    return np.nan

In [18]:
def check_binary_categorical(df_oh, df_cf, df_fc, cat_feats, num_feats, model, converter):
    df_cf_found, df_fc_found = get_cleaned_processed(df_cf, df_fc, cat_feats, model, converter)
        
    if df_cf_found.shape[0] > 0:
        
        if len(cat_feats) > 0:
        
            num_feats_idx = [str(list(df_oh.columns).index(n_f)) for n_f in num_feats]

            cat_feats_idx = list(set(df_cf_found.columns)-set(num_feats_idx))

            factual_cat_cols = df_fc_found.loc[:,cat_feats_idx].apply(lambda x: x.map(lambda y: y not in [1.0, 0.0, 1, 0]), axis=1).T.sum()

            assert factual_cat_cols.sum() == 0

            output = df_cf_found.loc[:,cat_feats_idx].apply(lambda x: x.map(lambda y: y in [1.0, 0.0, 1, 0]), axis=1).apply(sum, axis=1).map(lambda x: int(x == len(cat_feats_idx)))
            output.index = df_cf_found.index
            
            return output
        
    return np.nan

In [19]:
def get_bin_feats_idx(df_oh, converter):
    bin_cols = converter.binary_cats
    bin_feats = []
    for b_c in bin_cols:
        try:
            bin_feats.append(list(df_oh.columns).index(b_c))
        except ValueError:
            bin_feats.append(list(df_oh.columns).index('0'*(4-len(b_c))+b_c))
    return bin_feats

In [ ]:
# General Scores

In [21]:
def md(df_oh, df_cf, df_fc, cat_feats, num_feats, model, converter):
    df_cf_found, df_fc_found = get_cleaned_processed(df_cf, df_fc, cat_feats, model, converter)
    
    if df_cf_found.shape[0] > 0:
        output_result = []
        for idx in range(df_cf_found.shape[0]):
            m_dis = sp.spatial.distance.mahalanobis(df_cf_found.iloc[idx].to_numpy(), df_fc_found.drop(columns=['output']).iloc[idx].to_numpy(), df_oh.drop(columns=['output']).cov().to_numpy())
            output_result.append(m_dis)
        return sum(output_result)/len(output_result)
    
    return np.nan

In [22]:
# Mean can give misleading values because very high values on MAD
def madd(df_oh, df_cf, df_fc, cat_feats, num_feats, model, converter):
    df_cf_found, df_fc_found = get_cleaned_processed(df_cf, df_fc, cat_feats, model, converter)
    
    if df_cf_found.shape[0] > 0:
        df_mad = df_cf_found.iloc[:0]
        
        df_oh_c = df_oh.copy()
        df_oh_c.columns = df_fc_found.columns
        num_feats_idx = [str(list(df_oh.columns).index(n_f)) for n_f in num_feats]
        
        cat_feats_idx = list(set(df_cf_found.columns)-set(num_feats_idx))

        mad_num = {}
        for n_feat_idx in num_feats_idx:
            # 1e-8 added to avoid 0 and, then, division by zero
            mad_num[n_feat_idx] = sp.stats.median_abs_deviation(df_oh_c[n_feat_idx])+1e-8
            
            df_mad[n_feat_idx] = abs(df_cf_found[n_feat_idx]-df_fc_found[n_feat_idx])/mad_num[n_feat_idx]
        
        for c_feat_idx in cat_feats_idx:
            df_mad[c_feat_idx] = (df_cf_found[c_feat_idx]!=df_fc_found[c_feat_idx]).map(int)
        
        output_result = 0
        if len(cat_feats_idx) > 0 :
            output_result += df_mad[cat_feats_idx].mean(axis=1).mean()
        if len(num_feats_idx) > 0 :
            output_result += df_mad[num_feats_idx].mean(axis=1).mean()
            
        return output_result
    
    return np.nan

In [23]:
def get_cleaned_processed(df_cf, df_fc, cat_feats, model, converter):
    df_fc_pro = get_df_fc_pro(df_cf, df_fc, cat_feats, converter)
    df_cf_c, df_fc_pro_c = get_correct(df_cf, df_fc_pro, model)
    
    return df_cf_c, df_fc_pro_c

In [24]:
def get_correct(df_cf, df_fc, model):
    df_cf_found, df_fc_found = remove_na(df_cf, df_fc)
    if df_cf_found.shape[0] > 0:
        labels_cf = model.predict(df_cf_found.to_numpy()).round()
        return df_cf_found[labels_cf.reshape(-1,1)!=df_fc_found['output'].to_numpy().reshape(-1,1)], df_fc_found[labels_cf.reshape(-1,1)!=df_fc_found['output'].to_numpy().reshape(-1,1)]
    
    return df_cf_found.iloc[:0], df_fc.iloc[:0]

In [25]:
def validity_total(df_cf, df_fc, model):
    df_cf_found, df_fc_found = remove_na(df_cf, df_fc)
    if df_cf_found.shape[0] > 0:
        return (model.predict(df_cf_found.to_numpy()).round().reshape(1,-1)!=df_fc_found['output'].to_numpy()).sum()/df_fc.shape[0]
    return 0.0

In [26]:
def validity_found(df_cf, df_fc, model):
    df_cf_found, df_fc_found = remove_na(df_cf, df_fc)
    if df_cf_found.shape[0] > 0:
        return (model.predict(df_cf_found.to_numpy()).round().reshape(1,-1)!=df_fc_found['output'].to_numpy()).sum()/df_fc_found.shape[0]
    return 0.0

In [27]:
def remove_na(df_cf, df_fc):
    return df_cf.dropna(), df_fc[0==df_cf.isna().sum(axis=1)]

In [28]:
def get_df_fc_pro(df_cf, df_fc, cat_feats, converter):
    array_df_fc_pro = []
    if len(cat_feats) > 0:
        for idx, row in df_fc.iterrows():
            row_fc_pro = converter.convert_to_oh(pd.DataFrame(row).T.drop(columns=['output']).to_numpy().tolist()[0])
            array_df_fc_pro.append(row_fc_pro)
        df_fc_pro = pd.DataFrame(array_df_fc_pro)
        df_fc_pro['output'] = df_fc['output']
    else:
        df_fc_pro = df_fc.copy()
        
    df_fc_pro.columns = list(df_cf.columns)+['output']
    
    return df_fc_pro


In [29]:
def sparsity(df_cf, df_fc, cat_feats, model, converter):
    df_cf_found, df_fc_found = get_cleaned_processed(df_cf, df_fc, cat_feats, model, converter)
    
    if df_cf_found.shape[0] > 0:
        df_fc_found = df_fc_found.copy().drop(columns=['output'])
        
        scores = []
        for i in range(df_cf_found.shape[0]):
            scores.append((df_cf_found.iloc[i].round(4)==df_fc_found.iloc[i].round(4)).sum()/df_cf_found.shape[1])

        return sum(scores)/len(scores)
    
    return np.nan
    

In [30]:
def l2(df_cf, df_fc, cat_feats, model, converter):
    df_cf_found, df_fc_found = get_cleaned_processed(df_cf, df_fc, cat_feats, model, converter)
    
    
    if df_cf_found.shape[0] > 0:
        df_fc_found = df_fc_found.copy().drop(columns=['output'])
        scores = []
        for i in range(df_cf_found.shape[0]):
            scores.append(np.linalg.norm(
                df_cf_found.iloc[i].round(4).to_numpy().reshape(-1,1)-
                df_fc_found.iloc[i].round(4).to_numpy().reshape(-1,1)))

        return sum(scores)/len(scores)
    
    return np.nan
        

In [31]:
cat_ds = []
num_ds = []
mix_ds = []

for dsName, dsData in VAR_TYPES.items():
    
    if len(dsData['categorical']) > 0 and len(dsData['numerical']):
        mix_ds.append(dsName)
    elif len(dsData['categorical']) > 0:
        cat_ds.append(dsName)
    else:
        num_ds.append(dsName)

In [32]:
onlyfiles = [f for f in listdir('./cfoutput') if isfile(join('./cfoutput', f))]

In [42]:
full_data = []
for file in onlyfiles:
    if file.split('_')[0] != 'TIME':
        n = file.split('_')[0]
        dsName = file.split('_')[1]
        framework = file.split('_')[2].split('.')[0]
        
        cat_feats = VAR_TYPES[dsName]['categorical']
        num_feats = VAR_TYPES[dsName]['numerical']
        
        model = load_model(f'./models/{n}_{dsName}.h5')
        
        if len(num_feats) > 0:
            df = pd.read_csv(f'./data/NORM_{dsName}.csv')
        else:
            df = pd.read_csv(f'./data/{dsName}.csv')
        
        if len(cat_feats) > 0 and len(num_feats) > 0:
            df_oh = pd.read_csv(f'./data/OH_NORM_{dsName}.csv')
        elif len(cat_feats) > 0:
            df_oh = pd.read_csv(f'./data/OH_{dsName}.csv')
        else:
            df_oh = pd.read_csv(f'./data/NORM_{dsName}.csv')
        
        df_cf = pd.read_csv(f'./cfoutput/{n}_{dsName}_{framework}.csv')
        df_train = pd.read_csv(f'./experiments_data/{n}_{dsName}_TRAINDATASET.csv')
        df_fc = pd.read_csv(f'./experiments_data/{n}_{dsName}_CFDATASET.csv').drop(columns=['Unnamed: 0'])
        
        converter = OHConverter.Converter(df, cat_feats, list(df_oh.columns))
        
        print(n, dsName, framework)
        
        scoreValidityTotal = validity_total(df_cf, df_fc, model)
        scoreValidityFound = validity_found(df_cf, df_fc, model)
        scoreSparsity = sparsity(df_cf, df_fc, cat_feats, model, converter)
        scoreL2 = l2(df_cf, df_fc, cat_feats, model, converter)
        scoreMadd = madd(df_oh, df_cf, df_fc, cat_feats, num_feats, model, converter)
        scoreMd = md(df_oh, df_cf, df_fc, cat_feats, num_feats, model, converter)
        
        df_cf_found, df_fc_found = get_cleaned_processed(df_cf, df_fc, cat_feats, model, converter)
        
        ruc = []
        rmc = []
        
        scoreCheckBinCat = check_binary_categorical(df_oh, df_cf, df_fc, cat_feats, num_feats, model, converter)
        if scoreCheckBinCat is not np.nan:
            ruc.append(scoreCheckBinCat)
        
        scoreCheckOutsideNumRange = check_outside_num_range(dsName, df_oh, df_cf, df_fc, cat_feats, num_feats, model, converter)
        if scoreCheckOutsideNumRange is not np.nan:
            ruc.append(scoreCheckOutsideNumRange)
        
        scoreCheckOneHotIntegrity = check_one_hot_integrity(df_oh, df_cf, df_fc, cat_feats, num_feats, model, converter)
        if scoreCheckOneHotIntegrity is not np.nan:
            rmc.append(scoreCheckOneHotIntegrity)
            
        
        if dsName in list(dict_functions.keys()):
            custom_ruc_functions = dict_functions[dsName]['ruc']
            custom_rmc_functions = dict_functions[dsName]['rmc']
            
            for c_ruc_function in custom_ruc_functions:
                output_ruc = c_ruc_function(dsName, df_oh, df_cf, df_fc, cat_feats, num_feats, model, converter)
                if output_ruc is not np.nan:
                    ruc.append(output_ruc)
                else:
                    assert len(ruc) == 0
            
            for c_rmc_function in custom_rmc_functions:
                output_rmc = c_rmc_function(dsName, df_oh, df_cf, df_fc, cat_feats, num_feats, model, converter)
                if output_rmc is not np.nan:
                    rmc.append(output_rmc)
                else:
                    assert len(rmc) == 0
        
        if len(ruc) > 0:
            scoreRUC = (sum(ruc)/len(ruc)==1).map(int).sum()/df_cf.shape[0]
        else:
            scoreRUC = np.nan
        
        if len(rmc) > 0:
            scoreRMC = (sum(rmc)/len(rmc)==1).map(int).sum()/df_cf.shape[0]
        else:
            scoreRMC = np.nan
            
        full_data.append([dsName, n, framework, scoreValidityTotal, scoreValidityFound, scoreSparsity, scoreL2, scoreRUC, scoreRMC, scoreMadd, scoreMd, df_cf.shape[0], df_cf_found.shape[0]])

1 StatlogGC SEDC


/home/rmazzine/anaconda3/envs/jupyterlab-debugger/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator StandardScaler from version 0.23.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


1 SDD ALIBIC
3 Lymphography ALIBIC
0 Ecoli ALIBIC


/home/rmazzine/anaconda3/envs/jupyterlab-debugger/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator StandardScaler from version 0.23.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


0 SoybeanSmall SEDC
0 BalanceScale ALIBIC
0 Ecoli SEDC


...

1 Wine SEDC


/home/rmazzine/anaconda3/envs/jupyterlab-debugger/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator StandardScaler from version 0.23.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


1 Chess SEDC
1 Iris SEDC


/home/rmazzine/anaconda3/envs/jupyterlab-debugger/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator StandardScaler from version 0.23.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


0 StatlogGC ALIBIC
1 DefaultOfCCC ALIBIC


In [43]:
df_out = pd.DataFrame(full_data, columns=['dsName', 'n', 'framework', 'ValidityTotal', 'ValidityFound', 'Sparsity', 'L2', 'RUC', 'RMC', 'MADD', 'MD', 'total', 'found'])

In [44]:
df_out.sort_values(by=['dsName', 'n']).head(50)

,dsName,n,framework,ValidityTotal,ValidityFound,Sparsity,L2,RUC,RMC,MADD,MD,total,found
17,Adult,0,ALIBIC,0.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,10,0
56,Adult,0,SEDC,1.0,1.000000,0.875701,2.436646,0.0,0.1,2.443201e+05,1.764372,10,10
29,Adult,1,ALIBIC,0.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,10,0
40,Adult,1,SEDC,1.0,1.000000,0.845794,2.320825,0.1,0.1,1.961083e+07,2.223861,10,10
13,BCW,0,SEDC,1.0,1.000000,0.506250,4.033521,1.0,1.0,7.173305e-01,7.211506,10,10
65,BCW,0,ALIBIC,0.1,0.125000,0.687500,0.660435,0.1,0.1,8.246770e-02,0.677556,10,1
49,BCW,1,SEDC,1.0,1.000000,0.484375,4.006548,1.0,1.0,7.372488e-01,7.231116,10,10
66,BCW,1,ALIBIC,0.1,0.166667,0.937500,1.142192,0.1,0.1,6.089790e-02,1.203916,10,1
5,BalanceScale,0,ALIBIC,0.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,10,0
16,BalanceScale,0,SEDC,0.5,0.714286,0.860000,1.648528,0.5,0.0,1.400000e-01,0.659939,10,5


In [135]:
aggregated_scores = []
for dsName in df_out['dsName'].unique():
    for framework in df_out['framework'].unique():
        
        setDf = df_out[(df_out['framework']==framework)&(df_out['dsName']==dsName)&(df_out['found']>0)].copy()
        
        data_info = pd.Series({'dsName': dsName, 'framework': framework})
        
        if setDf['found'].sum() > 0:

            scoresDfTotal = (setDf[['ValidityTotal']]*setDf['total'].values.reshape(-1,1)).sum()/setDf['total'].sum()
            scoresDfFound = (setDf[['ValidityFound', 'Sparsity', 'L2', 'RUC', 'RMC', 'MADD', 'MD']]*setDf['found'].values.reshape(-1,1)).sum()/setDf['found'].sum()
            
            scores = pd.concat([data_info, scoresDfTotal, scoresDfFound])
        else:
            scores = pd.Series({'ValidityFound': 0.0, 'Sparsity': np.nan, 'L2': np.nan, 'RUC': np.nan, 'RMC': np.nan, 'MADD': np.nan, 'MD': np.nan, 'ValidityTotal': 0.0})
            scores = pd.concat([data_info, scores])
            
        aggregated_scores.append(scores)

In [137]:
pd.DataFrame(aggregated_scores)

,dsName,framework,ValidityTotal,ValidityFound,Sparsity,L2,RUC,RMC,MADD,MD
0,StatlogGC,SEDC,1.000000,1.000000,0.679661,2.618176,0.000000,0.0000,8.316505e+06,2.249368
1,StatlogGC,ALIBIC,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
2,SDD,SEDC,1.000000,1.000000,0.614583,3.292288,1.000000,0.0000,5.744649e-01,4.684171
3,SDD,ALIBIC,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
4,Lymphography,SEDC,1.000000,1.000000,0.826000,2.939757,1.000000,0.0000,1.740000e-01,1.567435
5,Lymphography,ALIBIC,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
6,Ecoli,SEDC,0.850000,1.000000,0.445378,1.867708,0.876471,0.0000,1.032833e+06,2.431174
7,Ecoli,ALIBIC,0.100000,0.208333,0.500000,1.771466,0.000000,0.0000,2.242393e+06,2.112248
8,SoybeanSmall,SEDC,0.900000,1.000000,0.852166,2.949042,0.911111,0.0000,1.478343e-01,2.369782
9,SoybeanSmall,ALIBIC,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
